In [196]:
import pandas as pd
df = pd.read_csv('cats_worldwide.csv')
data = df.to_dict('records') 
len(data)

23

In [197]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [198]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

In [199]:
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

In [200]:
qdrant.recreate_collection(
    collection_name="my_cats",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

/var/folders/2b/83nz7_sj2nq69nb2td6lz6080000gp/T/ipykernel_4185/1388995230.py:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [201]:
# vectorize!
qdrant.upload_points(
    collection_name="my_cats",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["notes"]).tolist(),
            payload=doc,
        ) for idx, doc in enumerate(data) # data is the variable holding all the cats
    ]
)

In [202]:
user_prompt = "Find a white cat who likes to eat fish and sleep"

In [203]:
hits = qdrant.search(
    collection_name="my_cats",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'name': 'Youzi', 'region': 'Hangzhou,China', 'breed': 'Chinese Garden Cat', 'age': 5, 'notes': 'A very clingy farm cat that loves its mom. Its fur is entirely white. It sleeps almost 24 hours a day. It only likes to eat salmon-flavored canned food.'} score: 0.6854454129730825
{'name': 'Daisy', 'region': 'Hangzhou,China', 'breed': 'British Short Hair', 'age': 1, 'notes': 'A sweet and gentle cat that loves to cuddle. Its fur is white with gray spots. It enjoys being petted and is very affectionate towards its owner.'} score: 0.5272890148383473
{'name': 'Lucy', 'region': 'Hangzhou,China', 'breed': 'Chinese Garden Cat', 'age': 3, 'notes': 'A sweet and gentle cat that loves to cuddle. Its fur is white with gray spots. It enjoys being petted and is very affectionate towards its owner.'} score: 0.5272890148383473


In [204]:
search_results = [hit.payload for hit in hits]

In [ ]:
from openai import OpenAI
client = OpenAI(
    base_url="http://127.0.0.1:8080/v1", 
    api_key = "sk-no-key-required"
)
completion = client.chat.completions.create(
    model="llava-v1.5-7b-Q4_K.gguf",
    messages=[
        {"role": "system", "content": """You are chatbot, a cat specialist. 
        Your top priority is to help users find one cat based on their request. 
        Include phrases like “I found xxx” and then includeALL THE DETAILED INFORMATION about the cat. """},
        {"role": "user", "content": user_prompt},
        # Add the retrieved cat information (search_results) as the assistant’s previous reply to the conversation history
        {"role": "assistant", "content": str(search_results)}
    ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content='I found Youzi, a 5-year-old Chinese Garden Cat from Hangzhou, China. Its fur is entirely white, and it sleeps almost 24 hours a day. It only likes to eat salmon-flavored canned food.\n', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None)
